In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from datetime import datetime
import time
import yfinance as yf

In [2]:
tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]['Symbol']

In [29]:
yearly_combined = pd.DataFrame()

for ticker in tickers[0:2]:
    start = time.time()
    try:
        # get stock info
        stock = yf.Ticker(ticker)
        info = stock.info
        
        balance = stock.balance_sheet
        financials = stock.financials
        earnings = stock.earnings
        cashflow = stock.cashflow
        
        # format date
        start_dates = balance.columns.values
        end_dates = start_dates + np.timedelta64(366, 'D')
        start_dates = [str(date)[:-19] for date in start_dates]
        end_dates = [str(date)[:-19] for date in end_dates]
        
        # get yearly data to fix issues with days when market closed
        year1 = stock.history(start=start_dates[0], end=end_dates[0], period='day').reset_index(drop=True)
        year2 = stock.history(start=start_dates[1], end=end_dates[1], period='day').reset_index(drop=True)
        year3 = stock.history(start=start_dates[2], end=end_dates[2], period='day').reset_index(drop=True)
        year4 = stock.history(start=start_dates[3], end=end_dates[3], period='day').reset_index(drop=True)
        
        # format arrays
        columns = earnings.T.columns.values
        balance.columns = columns
        financials.columns = columns
        earnings = earnings.T
        cashflow.columns = columns
        cashflow = cashflow.drop(['Net Income'], axis=0)   #financials has net income. No need for collision
        yearly = pd.concat([balance, financials, cashflow, earnings]).T
        columns = yearly.columns.values
        
        # Price
        shareprice = np.array([year4.loc[0, 'Close'], year3.loc[0, 'Close'], year2.loc[0, 'Close'],
                               year1.loc[0, 'Close']])
        
        # indicators
         # liquidity
        yearly['current_ratio'] = yearly['Total Current Assets'] / yearly['Total Current Liabilities']
        
        yearly['quick_ratio'] = yearly['Total Current Assets']
        if 'Inventory' in columns:
            yearly['quick_ratio'] += yearly['Inventory']
        yearly['quick_ratio'] /= yearly['Total Current Liabilities']
        
        yearly['cash_ratio'] = yearly['Cash']
        if 'Short Term Investments' in columns:
            yearly['cash_ratio'] += yearly['Short Term Investments']
        yearly['cash_ratio'] /= yearly['Total Current Liabilities']
        
        yearly['operating_cashflow'] = yearly['Total Cash From Operating Activities'] / yearly['Total Current Liabilities']
        
        yearly['equity_ratio'] = yearly['Total Stockholder Equity'] / yearly['Total Assets']
        
         # Leverage
        yearly['debt_to_equity'] = yearly['Total Liab'] / yearly['Total Stockholder Equity']
        
        yearly['debt_to_assets'] = yearly['Total Liab'] / yearly['Total Assets']
        
        yearly['interest_coverage'] = yearly['Operating Income'] / yearly['Interest Expense']
        
         # Efficiencies
        yearly['asset_turnover'] = yearly['Revenue'] / yearly["Total Assets"]
        
        if 'Net Receivables' in columns:
            yearly['receivables_turnover'] = yearly['Revenue'] / yearly['Net Receivables']
        else:
            yearly['receivables_turnover'] = np.nan
        
         # profitability
        yearly['gross_margin'] = yearly['Gross Profit'] / yearly['Revenue']
        
        yearly['profit_margin'] = yearly['Net Income'] / yearly['Revenue']
        
        yearly['operating_margin'] = yearly['Operating Income'] / yearly['Revenue']
        
        yearly['return_on_equity'] = yearly['Net Income'] / yearly['Total Stockholder Equity']
        
        yearly['return_on_assets'] = yearly['Net Income'] / yearly['Total Assets']
        
        if 'Issuance of Stock' in columns:
            yearly['eps'] = yeraly['Earnings'] / yearly['Issuance Of Stock']
        else:
            yearly['eps'] = yearly['Earnings'] / info['sharesOutstanding']
        
        yearly['p/e'] = shareprice / yearly['eps']
  
        yearly['ticker'] = ticker
        yearly['sector'] = info['sector']
        yearly['year'] = yearly.index.values
        
        # percent change
        startprice = np.array([year4.loc[1, 'Open'], year3.loc[1, 'Open'], 
                               year2.loc[1, 'Open'], year1.loc[1, 'Open']])
        endprice = np.array([year4.loc[len(year4)-1, 'Close'], year3.loc[len(year3)-1, 'Close'], 
                             year2.loc[len(year2)-1, 'Close'], year1.loc[len(year1)-1, 'Close']])
        pctdiff = (endprice-startprice)/startprice
        yearly['result'] = pctdiff
        
        columns = ['ticker', 'year', 'sector', 'current_ratio', 'quick_ratio', 'cash_ratio', 'operating_cashflow', 
                   'equity_ratio', 'debt_to_equity', 'debt_to_assets', 'interest_coverage', 'asset_turnover', 
                   'receivables_turnover', 'gross_margin', 'profit_margin', 'operating_margin', 'return_on_equity', 
                  'return_on_assets', 'eps', 'p/e', 'result']
        
        yearly = yearly[columns].reset_index(drop=True)
        
        if yearly_combined.empty:
            yearly_combined = yearly
        else:
            yearly_combined = pd.concat([yearly_combined, yearly]).reset_index(drop=True)
        
    except Exception as e:
        print(ticker, e)
    time_diff = time.time()-start
    if time_diff < 10:
        time.sleep(10-time_diff)
        
display(yearly_combined)
#yearly_combined.to_csv('sp500_data.csv', index=False)

,ticker,year,sector,current_ratio,quick_ratio,cash_ratio,operating_cashflow,equity_ratio,debt_to_equity,debt_to_assets,...,asset_turnover,receivables_turnover,gross_margin,profit_margin,operating_margin,return_on_equity,return_on_assets,eps,p/e,result
0,MMM,2018,Industrials,1.70481,2.25656,0.527393,0.825014,0.319638,2.12382,0.678854,...,0.696061,6.86897,0.505997,0.180711,0.237754,0.393527,0.125786,9.39973,17.9674,-0.029047
1,MMM,2019,Industrials,1.885,2.41834,0.63387,1.02076,0.271777,2.67452,0.726871,...,0.678777,6.65342,0.488082,0.169561,0.222679,0.423486,0.115094,7.93767,20.2958,0.030606
2,MMM,2020,Industrials,1.40653,1.8548,0.265777,0.766645,0.22533,3.43168,0.77326,...,0.720661,6.48479,0.468307,0.140349,0.195905,0.448872,0.101144,9.57546,17.2802,0.052540
3,MMM,2021,Industrials,1.89246,2.49517,0.4463,0.888874,0.268384,2.7207,0.730192,...,0.96863,6.90123,0.455664,0.151294,0.219686,0.546039,0.146548,10.4049,16.7401,-0.141099
4,AOS,2018,Industrials,1.5665,1.96666,0.564355,0.573025,0.527343,0.8963,0.472657,...,0.91754,5.02506,0.411211,0.152797,0.191003,0.265855,0.140197,3.41598,11.7237,0.149682
5,AOS,2019,Industrials,1.82557,2.16417,0.778066,0.63421,0.584776,0.710058,0.415224,...,0.946847,5.11573,0.370301,0.115247,0.147359,0.186604,0.109121,2.84537,15.9723,0.173245
6,AOS,2020,Industrials,1.95747,2.35277,0.719374,0.595173,0.545062,0.834653,0.454938,...,0.946795,4.91145,0.407799,0.127793,0.158775,0.221982,0.120994,2.65234,20.1158,0.597737
7,AOS,2021,Industrials,2.08646,2.47447,0.821342,0.571629,0.55901,0.788876,0.44099,...,1.15217,5.46717,0.3689,0.125519,0.1555,0.258707,0.14462,3.74589,22.5839,-0.274972
